In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: MacOSX


In [11]:
# Take the values from variations_start_stop.txt and consolidate tha ranges and put this into variable_regions.txt

# def merge_intervals(times):
#     saved = list(times[0])
#     for st, en in sorted([sorted(t) for t in times]):
#         if st <= saved[1]:
#             saved[1] = max(saved[1], en)
#         else:
#             yield tuple(saved)
#             saved[0] = st
#             saved[1] = en
#     yield tuple(saved)

def merge_intervals(intervals):
    #sort the intervals
    sorted_by_lower_bound = sorted(intervals, key=lambda tup: tup[0])
    merged = []

    for higher in sorted_by_lower_bound:
        if not merged:
            merged.append(higher)
        else:
            lower = merged[-1]
            # test for intersection between lower and higher:
            # we know via sorting that lower[0] <= higher[0]
            if higher[0] <= lower[1]:
                upper_bound = max(lower[1], higher[1])
                merged[-1] = (lower[0], upper_bound)  # replace by merged interval
            else:
                merged.append(higher)
    return merged


# read the text file
df = pd.read_csv('variations_start_stop.txt', sep='\t')
all_chr = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
           '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y']

for chrm in all_chr:

    # choose chromosomes only in all_chr list since there maybe others
    to_keep = [chrm]
    # only keep rows that contain the string in to_keep
    df_subset = df[df['chr'].isin(to_keep)]
    # create intervals of the form: [(5, 7), (11, 116), (3, 4), (10, 12), (7, 8), (6, 12)]
    intervals_list = []
    for i in xrange(len(df_subset)):
        intervals_list.append((df_subset.iloc[i,1], df_subset.iloc[i,2]))
#     print "done creating intervals"
    # runt the function to get the merged values
    merged_list = merge_intervals(intervals_list)
#     print "done merging intervals"
    sorted_merged_list = sorted(merged_list, key=lambda tup: tup[0])
#     print "done sorting intervals"
    with open('variable_regions.txt', 'ab') as vr_file:
        for vr in sorted_merged_list:
            string = str(chrm) + '\t' + str(vr[0]) + '\t' + str(vr[1]) + '\n'
            vr_file.write(string)
    print "Finished Chromosome" + str(chrm)

Finished Chromosome1


In [9]:
# take the values from variable_regions.txt and create non_variable_regions.txt where the NVR file is disjoint

all_chr = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
           '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y']
all_sizes = [248956422, 242193529, 198295559, 190214555, 181538259, 170805979,
             159345973, 145138636, 138394717, 133797422, 135086622, 133275309,
             114364328, 107043718, 101991189, 90338345, 83257441, 80373285,
             58617616, 64444167, 46709983, 50818468, 156040895, 57227415]

df = pd.read_csv('variable_regions.txt', sep='\t', names=['chr', 'start', 'stop'], header=None)

for chrm, end in zip(all_chr, all_sizes):

    start = 1
    non_variable_regions = []

    to_keep = [chrm]
    # only keep rows that contain the string in to_keep
    df2 = df[df['chr'].isin(to_keep)]
    for i in xrange(len(df2)):
        # calculate the stop point, it is the base pair just before the start in the VR
        stop = df2.iloc[i, 1] - 1
        if start <= stop:
            non_variable_regions.append([start, stop])
        # calculate the start point, it is the base pair just after the stop in the VR
        start = df2.iloc[i, 2] + 1

    # need to determine if the last stop point is less than or equal to the total # of base pairs
    # if it is then this is a new NVR
    if start <= end:
        non_variable_regions.append([start, end])

    # save the file
    with open('a_non_variable_regions.txt', 'ab') as myfile:
        for non_variation in non_variable_regions:
            string = chrm + '\t' + str(non_variation[0]) + '\t' + str(non_variation[1]) + '\n'
            myfile.write(string)
    print "Finished Chromosome" + str(chrm)
    
df = pd.read_csv('a_non_variable_regions.txt', sep='\t', header=None, names=['chr', 'start', 'stop'])
nvr_bp = []
for chrm in all_chr:
    to_keep = [chrm]
    # only keep rows that contain the string in to_keep
    df2 = df[df['chr'].isin(to_keep)]
    bp = sum(df2['stop'] - df2['start'] + 1)
    nvr_bp.append(bp)

N = len(nvr_bp)
x = range(N)
width = 1/1.5

plt.bar(x, nvr_bp, width, color="blue")
plt.xlabel( 'Chromosome' )
plt.ylabel( 'Size (Number of Base Pairs)' )
plt.show()

Finished Chromosome1
Finished Chromosome2
Finished Chromosome3
Finished Chromosome4
Finished Chromosome5
Finished Chromosome6
Finished Chromosome7
Finished Chromosome8
Finished Chromosome9
Finished Chromosome10
Finished Chromosome11
Finished Chromosome12
Finished Chromosome13
Finished Chromosome14
Finished Chromosome15
Finished Chromosome16
Finished Chromosome17
Finished Chromosome18
Finished Chromosome19
Finished Chromosome20
Finished Chromosome21
Finished Chromosome22
Finished ChromosomeX
Finished ChromosomeY


In [10]:
# sanity check
all_sizes = [248956422, 242193529, 198295559, 190214555, 181538259, 170805979,
             159345973, 145138636, 138394717, 133797422, 135086622, 133275309,
             114364328, 107043718, 101991189, 90338345, 83257441, 80373285,
             58617616, 64444167, 46709983, 50818468, 156040895, 57227415]
total_bp = sum(all_sizes)
df1 = pd.read_csv('variable_regions.txt', sep='\t', names=['chr', 'start', 'stop'], header=None)
df2 = pd.read_csv('non_variable_regions.txt', sep='\t', names=['chr', 'start', 'stop'], header=None)
vr_bp = sum(df1['stop'] - df1['start'] + 1)
nvr_bp = sum(df2['stop'] - df2['start'] + 1)
print total_bp
print vr_bp + nvr_bp


# df = pd.read_csv('variable_regions.txt', sep='\t', header=None, names=['chr', 'start', 'stop'])
# for i in xrange(len(df)):
#     if (df.iloc[i,2] - df.iloc[i,1] + 1) == 0:
#         print i

3088269832
3088269832
